In [32]:
import javalang
from codegraph.java_parser_copy import _anno_details, _extract_http_basic, _anno_kv


src = r'''
package com.example.demo.controllers;
import org.springframework.stereotype.Controller;
import org.springframework.web.bind.annotation.*;
@Controller
public class ProductController {
  @PostMapping("/addingProduct")
  public String addProduct(@ModelAttribute Product product) { return "ok"; }
  @GetMapping("/updatingProduct/{productId}")
  public String updateProduct(@ModelAttribute Product product, @PathVariable("productId") int id) { return "ok"; }
  @GetMapping("/deleteProduct/{productId}")
  public String delete(@PathVariable("productId") int id) { return "ok"; }
}
'''
tree = javalang.parse.parse(src)
for t in tree.types:
    for m in t.methods:
        md = [_anno_details(a) for a in (m.annotations or [])]
        print(m.name, md, _extract_http_basic(md))


addProduct [{'name': '@PostMapping', 'full': '@PostMapping(/addingProduct)', 'args': {'value_list': ['/addingProduct'], 'value': '/addingProduct'}}] {'methods': ['POST'], 'paths': ['/addingProduct'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}
updateProduct [{'name': '@GetMapping', 'full': '@GetMapping(/updatingProduct/{productId})', 'args': {'value_list': ['/updatingProduct/{productId}'], 'value': '/updatingProduct/{productId}'}}] {'methods': ['GET'], 'paths': ['/updatingProduct/{productId}'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}
delete [{'name': '@GetMapping', 'full': '@GetMapping(/deleteProduct/{productId})', 'args': {'value_list': ['/deleteProduct/{productId}'], 'value': '/deleteProduct/{productId}'}}] {'methods': ['GET'], 'paths': ['/deleteProduct/{productId}'], 'consumes': [], 'produces': [], 'params': [], 'headers': [], 'name': None}


In [18]:
import sys, inspect, importlib, javalang
import codegraph.java_parser as jp

print("javalang:", javalang.__version__)
print("USING PARSER FILE:", jp.__file__)
print("PYTHONPATH head:", sys.path[:3])

# Show the first lines of the helper you just edited
print("KV HEAD:", inspect.getsource(jp._anno_kv).splitlines()[:8])

# Optional: add a signature string in your file and read it
print("SIGNATURE:", getattr(jp, "SIGNATURE", "<none>"))


javalang: 0.13.0
USING PARSER FILE: /Users/saichaitanyadarla/Downloads/codegraph_inline_full/codegraph/java_parser.py
PYTHONPATH head: ['/Library/Frameworks/Python.framework/Versions/3.13/lib/python313.zip', '/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13', '/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload']


TokenError: ('EOF in multi-line string', (105, 5))

In [24]:
import javalang, sys
src = open("/Users/saichaitanyadarla/Documents/java/FoodFrenzy/src/main/java/com/example/demo/controllers/ProductController.java").read()
tree = javalang.parse.parse(src)

for t in tree.types:
    print("Type:", t.name)
    for m in getattr(t, "methods", []):
        print("  Method:", m.name)
        for a in (m.annotations or []):
            print("@" + getattr(a, "name", "?"),
                  "member=", getattr(a, "member", None),
                  "value=", getattr(a, "value", None),
                  "element=", getattr(a, "element", None))


Type: ProductController
  Method: addProduct
@PostMapping member= None value= None element= Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value="/addingProduct")
  Method: updateProduct
@GetMapping member= None value= None element= Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value="/updatingProduct/{productId}")
  Method: delete
@GetMapping member= None value= None element= Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value="/deleteProduct/{productId}")
